In [70]:
# Encoding text using bag of words
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [71]:
# Integer Encoding
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [72]:
# Sentiment Analysis
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

In [73]:
# Lets look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

In [74]:
# making train and test the same length
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [75]:
# Creating the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [76]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_6 (LSTM)               (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [77]:
# Train the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=5, validation_split=0.2)

Epoch 1/5
625/625 [==============================] - 11s 16ms/step - loss: 0.4283 - acc: 0.8067 - val_loss: 0.3356 - val_acc: 0.8592
Epoch 2/5
625/625 [==============================] - 9s 15ms/step - loss: 0.2427 - acc: 0.9038 - val_loss: 0.2842 - val_acc: 0.8856
Epoch 3/5
625/625 [==============================] - 10s 16ms/step - loss: 0.1803 - acc: 0.9334 - val_loss: 0.2849 - val_acc: 0.8860
Epoch 4/5
625/625 [==============================] - 10s 16ms/step - loss: 0.1495 - acc: 0.9463 - val_loss: 0.2875 - val_acc: 0.8840
Epoch 5/5
625/625 [==============================] - 10s 16ms/step - loss: 0.1290 - acc: 0.9547 - val_loss: 0.2961 - val_acc: 0.8936


In [78]:
# Evaluated data
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 5s 6ms/step - loss: 0.3771 - acc: 0.8628
[0.3770900368690491, 0.8627600073814392]


In [79]:
# making predictions, need to encode text to the same data as before
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [80]:
# while were at it lets make a decode function
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [81]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.89792436]
[0.2778695]


In [82]:
# RNN Play generator
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [83]:
# loading file
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [84]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [85]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [86]:
# Encoding
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [87]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [88]:
# convert numeric values to text
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print("Text:",int_to_text(text_as_int[:13]))
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [89]:
# Creating training examples
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [90]:
# turn stream of characters into batches of desired length
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [91]:
# split sequences into input and output
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [92]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [93]:
# Make training batches
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [94]:
# Building the model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_7 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_7 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [95]:
# Creating a Loss Function
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [96]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 1.6750925e-04  2.9949772e-03 -3.2634435e-03 ...  1.7700054e-03
   -3.3006400e-03 -4.2938464e-03]
  [ 2.0768566e-03  1.8246891e-03 -4.3698987e-03 ...  1.0308770e-03
   -3.0184514e-03 -4.7110617e-03]
  [ 3.1972053e-03  2.2145533e-03 -3.4424802e-03 ... -1.7368386e-03
   -9.2933560e-04  4.2157276e-03]
  ...
  [ 5.6856778e-03 -5.2796234e-03  2.9977770e-03 ... -2.8356989e-03
    6.4860755e-03 -1.4472140e-03]
  [ 2.3558913e-03  2.1647720e-04 -5.2193692e-04 ...  1.8250858e-03
    3.6065197e-03 -7.4210751e-05]
  [-2.7668229e-03  2.2691994e-03  8.2934210e-03 ...  5.8786459e-03
    8.1866619e-04  1.7116538e-03]]

 [[ 2.1273349e-03  1.6227949e-03  4.2960835e-03 ... -2.1441185e-03
    3.0003081e-05 -2.2111759e-03]
  [ 8.0583710e-04 -4.9071712e-04  6.7434581e-03 ... -4.7183684e-03
    2.3298408e-03  4.5099752e-03]
  [-4.2391475e-04 -1.7265291e-03  6.4124744e-03 ... -2.6627551e-03
    3.7124110e-03  2.0994756e-03]
  ...
  [ 8.3589442e-03  4.3370817e-03 -1.9613984e-03 ... -9.4750093e

In [97]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 1.6750925e-04  2.9949772e-03 -3.2634435e-03 ...  1.7700054e-03
  -3.3006400e-03 -4.2938464e-03]
 [ 2.0768566e-03  1.8246891e-03 -4.3698987e-03 ...  1.0308770e-03
  -3.0184514e-03 -4.7110617e-03]
 [ 3.1972053e-03  2.2145533e-03 -3.4424802e-03 ... -1.7368386e-03
  -9.2933560e-04  4.2157276e-03]
 ...
 [ 5.6856778e-03 -5.2796234e-03  2.9977770e-03 ... -2.8356989e-03
   6.4860755e-03 -1.4472140e-03]
 [ 2.3558913e-03  2.1647720e-04 -5.2193692e-04 ...  1.8250858e-03
   3.6065197e-03 -7.4210751e-05]
 [-2.7668229e-03  2.2691994e-03  8.2934210e-03 ...  5.8786459e-03
   8.1866619e-04  1.7116538e-03]], shape=(100, 65), dtype=float32)


In [98]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 0.00016751  0.00299498 -0.00326344  0.00203141 -0.00481923  0.00127132
  0.00220486 -0.00478442  0.0044174  -0.00327809 -0.00456909 -0.00178141
 -0.0017141  -0.00145762  0.00106    -0.00120031  0.00087855 -0.00044071
 -0.00281738  0.00176086 -0.00114317 -0.00384362 -0.00371323 -0.00146628
 -0.00493713 -0.00253771  0.00417714 -0.00778077 -0.0004183  -0.00093889
 -0.00159036  0.00322513  0.00437075  0.00309451  0.00100923 -0.00428651
 -0.00276882  0.00015249  0.00037626 -0.00214973 -0.00346406 -0.0027049
 -0.00339789 -0.00176939  0.00041107 -0.00330927  0.00178166 -0.00122692
  0.0024228   0.00012841 -0.00633711 -0.00027688  0.00639298 -0.00163627
  0.00114704 -0.00314441 -0.00043513  0.00133852  0.0068442   0.00102606
 -0.00621608  0.00230392  0.00177001 -0.00330064 -0.00429385], shape=(65,), dtype=float32)


In [99]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"rEB;TqsAnsWUlfGNZeTdQCeRJrMVDYv?NTmVizpA$lK xXlV- l'OSS\nQmqXN;xfcnq:mD!PxoMe\nzgeavuf&k!HLi,elgVdz$Hw"

In [100]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [101]:
# Compile the model
model.compile(optimizer='adam', loss=loss)

In [102]:
# Creating checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [103]:
# training the model
history = model.fit(data, epochs=5, callbacks=[checkpoint_callback])

Epoch 1/5
172/172 [==============================] - 11s 57ms/step - loss: 2.6286
Epoch 2/5
172/172 [==============================] - 10s 57ms/step - loss: 1.9013
Epoch 3/5
172/172 [==============================] - 10s 57ms/step - loss: 1.6484
Epoch 4/5
172/172 [==============================] - 10s 58ms/step - loss: 1.5170
Epoch 5/5
172/172 [==============================] - 10s 57ms/step - loss: 1.4366


In [104]:
# loading the model
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [105]:
# loading the model
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [109]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: 'tensorflow.python.util._pywrap_checkpoint_reader.C' object has no attribute 'endswith'

In [110]:
# Generating text
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [111]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

romeoth once, you for his mother's like
UScound all cortain with wise!
The trink that struck a maid they so the dward
and night behold you the ore of king!
But comes so fire your strenged but all ochain.

LADY ANNE:
You since this take you find Lord Henry?

GORZALO:
The lain o, a, the lack
Ahong revenge, he is you from her life:
If fire is forws, the earth levier the back.

KATHARINA:
Musingly look you; five, I'll take them father,
What I undill'd the slaughter'd thingsy
This world in duppion'd balliath'd:
This in dusy? Yet?

ANGBUL:
Why, sit it is me well: aly is the fire,
As bring for heart at as a miserion of like known'd
Than the loving digding that boy!
But it is my leave of Kings Richard,
If foul thee, Within couse to hel; I no, my marm rich to be marriang,
Who enepys have a
vising deadle
